### Start importing all libraries

In [33]:
# Global
import numpy as np
import tqdm

# Local
import examples_utils as eu
import fastPTA.utils as ut
import fastPTA.plotting_functions as pf

from fastPTA.priors import Priors
from fastPTA.signals import get_signal_model
from fastPTA.Fisher_code import compute_fisher
from fastPTA.MCMC_code import run_MCMC

### Constants to be used in the analysis

In [34]:
# Total observation time in years
T_obs_yrs = 10.0  # 6.03

# Number of frequencies used in the analysis
n_frequencies = 30

# Number of pulsars in the analysis
n_pulsars = 200

# Specify the type of noise to be used in the analysis
which_experiment = eu.mockSKA10

# Set the label to specify the signal model
signal_label = "power_law_SIGW"

# The analysis assumes a power law template, specify here the input parameters
log_amplitude = -7.1995  # log amplitude of the power law
tilt = 2.0  # Tilt of the power law
SIGW_log_amplitude = -1.7  # log amplitude SIGW
SIGW_log_width = np.log10(0.5)  # log width SIGW
SIGW_log_pivot = -8.25  # log pivot SIGW

# Specify the true signal parameters
signal_parameters = np.array(
    [log_amplitude, tilt, SIGW_log_amplitude, SIGW_log_width, SIGW_log_pivot]
)

# Specify the labels for the signal parameters
parameter_labels = []

# Specify the priors for the signal parameters (should be a dictionary, the
# parameter names are the keys and the values are the dictionaries with the
# prior information) if not specfied here will build from fisher results
priors_dictionary = {}

# Number of points to generate for the Fisher
len_fisher_data = int(1e4)

# Name of the pulsar catalogue
outname = "pulsar_configurations/SKA200p.txt"

### Some inputs for the MCMC

In [35]:
# Whether the MCMC should be rerun
rerun_MCMC = True

# Whether the MCMC data should be regenerated
regenerate_MCMC_data = True

# Whether to generate a data realization or take data at face value
realization = False

# Number of burnin steps for the MCMC
burnin_steps_default = 100

# Maximum number of iterations to get R close to 1
i_max_default = 100

# Accepted value for the Gellman-Rubin convergence criterion
R_convergence_default = 1e-1

# Criterion to be used for the convergence of the MCMC chains
R_criterion_default = "mean_squared"

# Number of MCMC steps in each iteration
MCMC_iteration_steps_default = 300

# Path to the MCMC_data
path_to_MCMC_data = "generated_data/MCMC_data_Pl+SIGW_200p.npz"

# Path to the MCMC chains
path_to_MCMC_chains = "generated_chains/MCMC_chains_Pl+SIGW_200p.npz"

### Set the inputs for the Fisher and MCMC analysis

In [36]:
# Get the signal model
signal_model = get_signal_model(signal_label)

# Dictionary with the kwargs to generete the pulsar catalogs
generate_catalog_kwargs = {
    "n_pulsars": n_pulsars,
    "save_catalog": True,
    **which_experiment,
}

# Dictionary with the kwargs to generate noise and orf tensors
get_tensors_kwargs = {
    "add_curn": False,
    "regenerate_catalog": True,
    "path_to_pulsar_catalog": outname,
}

# Dictionary with the kwargs for the Fisher matrix
fisher_kwargs = {
    "T_obs_yrs": T_obs_yrs,
    "n_frequencies": n_frequencies,
    "signal_model": signal_model,
    "signal_parameters": signal_parameters,
}

# Dictionary with the kwargs for the MCMC
MCMC_kwargs = {
    "regenerate_MCMC_data": regenerate_MCMC_data,
    "realization": realization,
    "path_to_MCMC_data": path_to_MCMC_data,
    "path_to_MCMC_chains": path_to_MCMC_chains,
    "i_max": i_max_default,
    "R_convergence": R_convergence_default,
    "R_criterion": R_criterion_default,
    "burnin_steps": burnin_steps_default,
    "MCMC_iteration_steps": MCMC_iteration_steps_default,
}

### Prepare the model to scan over

In [37]:
if "regenerate_catalog" in get_tensors_kwargs.keys():
    if get_tensors_kwargs["regenerate_catalog"]:
        rerun_MCMC = True

(
    frequency,
    signal_value,
    HD_functions_IJ,
    HD_coeffs,
    effective_noise,
    SNR,
    fisher,
) = compute_fisher(
    **fisher_kwargs,
    get_tensors_kwargs=get_tensors_kwargs,
    generate_catalog_kwargs=generate_catalog_kwargs,
)

covariance = ut.compute_inverse(fisher)
fisher_data = np.random.multivariate_normal(
    signal_parameters, covariance, size=len_fisher_data
)
errors = np.sqrt(np.diag(covariance))
print("Fisher errors", errors)

fisher_ranges = np.vstack(
    (signal_parameters - 5 * errors, signal_parameters + 5 * errors)
)

get_tensors_kwargs["regenerate_catalog"] = False

Fisher errors [0.01655999 0.07620764 0.06124139 0.06356644 0.05247935]


In [38]:
to_keep = []

for i in tqdm.tqdm(range(len(fisher_data))):

    PBH_abundance = signal_model.get_PBH_abundance(fisher_data[i])

    if PBH_abundance > 1 or np.isnan(PBH_abundance):
        to_keep.append(False)

    else:
        to_keep.append(True)

fisher_reject = fisher_data[to_keep]

  2%|▏         | 179/10000 [00:00<00:22, 444.68it/s]


KeyboardInterrupt: 

In [ ]:
mean = np.mean(fisher_reject, axis=0)
std = np.std(fisher_reject, axis=0)

In [ ]:
parameter_names = signal_model.parameter_names

ndims = len(parameter_names)
nwalkers = max(2 * ndims, 5)
initial = np.empty((nwalkers, ndims))

mins = np.array([s * 0.99 if s > 0 else s * 1.01 for s in signal_parameters])
maxs = np.array([s * 1.01 if s > 0 else s * 0.99 for s in signal_parameters])

i = 0
while i < nwalkers:
    new = np.random.uniform(mins, maxs)
    
    PBH_abundance = signal_model.get_PBH_abundance(new)

    if PBH_abundance > 1 or np.isnan(PBH_abundance):
        pass  # Do nothing
    else:
        initial[i] = new
        i += 1

In [ ]:
# If the priors dictionary is empty, build it from the Fisher results
if not priors_dictionary:

    mins = np.array([s * 0.9 if s > 0 else s * 1.1 for s in signal_parameters])
    maxs = np.array([s * 1.1 if s > 0 else s * 0.9 for s in signal_parameters])

    parameter_names = signal_model.parameter_names
    diffs = np.abs(np.diff(fisher_ranges, axis=0).squeeze())

    for i in range(len(parameter_names)):

        if i < 2:
            priors_dictionary[parameter_names[i]] = {
                "uniform": {"loc": fisher_ranges[0, i], "scale": diffs[i]}
            }

        else:
            priors_dictionary[parameter_names[i]] = {
                "uniform": {"loc": -10.0, "scale": 10.0}
            }

priors_dictionary["log_amplitude_scalar"] = {
    "uniform": {"loc": -3.0, "scale": 2.0}
}
priors_dictionary["log_width"] = {"uniform": {"loc": -1.8, "scale": 2.6}}
priors_dictionary["log_pivot"] = {"uniform": {"loc": -9., "scale": 2.}}

priors = Priors(
    priors_dictionary, get_PBH_abundance=signal_model.get_PBH_abundance, 
)

In [ ]:
try:
    if rerun_MCMC:
        raise FileNotFoundError("Flag forces MCMC chains regeneration")

    MCMC_results = np.load(path_to_MCMC_chains)
    MCMC_data = MCMC_results["samples"]
    pdfs = MCMC_results["pdfs"]

except FileNotFoundError:
    MCMC_data, pdfs = run_MCMC(
        priors,
        initial=initial,
        **fisher_kwargs,
        **MCMC_kwargs,
        get_tensors_kwargs=get_tensors_kwargs,
    )


Regenerating MCMC data
- Data will use the expectation value

Initial run


  0%|          | 0/100 [00:00<?, ?it/s]/home/mauro/.pyenv/versions/3.12.4/lib/python3.12/site-packages/emcee/moves/red_blue.py:99: RuntimeWarning: invalid value encountered in scalar subtract
  lnpdiff = f + nlp - state.log_prob[j]
100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


Burn-in dropped, here starts the proper run


100%|██████████| 300/300 [03:22<00:00,  1.48it/s]


At this step R = 66.1973


100%|██████████| 300/300 [03:20<00:00,  1.50it/s]


At this step R = 67.6183


 44%|████▎     | 131/300 [01:26<02:05,  1.35it/s]Traceback (most recent call last):
  File "/tmp/ipykernel_164329/797928998.py", line 3, in <module>
    raise FileNotFoundError("Flag forces MCMC chains regeneration")
FileNotFoundError: Flag forces MCMC chains regeneration

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/mauro/.pyenv/versions/3.12.4/lib/python3.12/site-packages/emcee/ensemble.py", line 640, in __call__
    return self.f(x, *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mauro/Media/HD1/My_projects/PTA_forecasts/code/fastpta_models/fastPTA/MCMC_code.py", line 352, in log_posterior
    if not jnp.isfinite(lp):
           ^^^^^^^^^^^^^^^^
  File "/home/mauro/.pyenv/versions/3.12.4/lib/python3.12/site-packages/jax/_src/array.py", line 293, in __bool__
    return bool(self._value)
                ^^^^^^^^^^^
  File "/home/mauro/.pyenv/versions/3.12.4/lib/python3.12/

emcee: Exception while calling your likelihood function:
  params: [-7.2127394   1.8291808  -1.75388935 -0.18919195 -8.12138234]
  args: [Array([3.16880878e-09, 6.33761756e-09, 9.50642634e-09, 1.26752351e-08,
       1.58440439e-08, 1.90128527e-08, 2.21816615e-08, 2.53504703e-08,
       2.85192790e-08, 3.16880878e-08, 3.48568966e-08, 3.80257054e-08,
       4.11945142e-08, 4.43633229e-08, 4.75321317e-08, 5.07009405e-08,
       5.38697493e-08, 5.70385581e-08, 6.02073668e-08, 6.33761756e-08,
       6.65449844e-08, 6.97137932e-08, 7.28826020e-08, 7.60514108e-08,
       7.92202195e-08, 8.23890283e-08, 8.55578371e-08, 8.87266459e-08,
       9.18954547e-08, 9.50642634e-08], dtype=float64, weak_type=True), <fastPTA.signals.Signal_model object at 0x7be419d8b920>, Array([[[ 0.4891426 , -0.02223717,  0.02765662, ...,  0.0669196 ,
         -0.04243413,  0.05342267],
        [-0.02223717,  0.56484644,  0.19322388, ..., -0.07917785,
         -0.05506931, -0.01595297],
        [ 0.02765662,  0.1932238

KeyboardInterrupt: 

In [ ]:
# Set the labels for the signal parameters
if not parameter_labels:
    parameter_labels = signal_model.parameter_labels

# Prepare input for the plotting function
datasets = [fisher_data, fisher_reject, MCMC_data]
weights = [
    np.ones(len_fisher_data),
    np.ones(len(fisher_reject)),
    np.ones(MCMC_data.shape[0]),  # type: ignore
]
smooth = [1.0, 1.0, 1.0]
ranges = np.array(fisher_ranges.T)
colors = [
    pf.my_colormap["red"],
    pf.my_colormap["blue"],
    pf.my_colormap["green"],
]

# Do the corner plot
pf.plot_corner(
    datasets,
    colors=colors,
    truths=signal_parameters,
    chain_labels=["Fisher", "Fisher reject", "MCMC"],
    weights=weights,
    smooth=smooth,
    labels=parameter_labels,
    # range=ranges,
    truth_color="black",
)